<a href="https://colab.research.google.com/github/axmadJ3/hh-notebook/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22hh_vacancy_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/HH_Vacancy/

In [ ]:
!pip install requests beautifulsoup4 urllib3 openpyxl tqdm selenium webdriver-manager

In [ ]:
# Скачиваем Google Chrome 115
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

# Устанавливаем скачанный .deb пакет
!dpkg -i google-chrome-stable_current_amd64.deb

!google-chrome-stable --version



In [ ]:
# Скачиваем ChromeDriver для соответствующей версии Google Chrome
!wget https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.85/linux64/chromedriver-linux64.zip

# Разархивируем файл
!unzip chromedriver-linux64.zip

# Проверяем содержимое распакованного архива (чтобы увидеть структуру папок)
!ls -l chromedriver-linux64

# Перемещаем ChromeDriver в нужную директорию (если папка вложенная)
!mv chromedriver-linux64/chromedriver /usr/local/bin/chromedriver

!which chromedriver



In [ ]:
# Перемещаем chromedriver из Google Drive в локальную файловую систему
!cp /content/drive/MyDrive/HH_Vacancy/chromedriver /usr/local/bin/chromedriver

# Устанавливаем права на выполнение для chromedriver
!chmod +x /usr/local/bin/chromedriver


НАЖМИТЕ ЗДЕСЬ ЧТОБ ВВЕСТИ BASE_URL

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import logging
import os
from google.colab import drive

# Монтируем Google Drive
drive.mount('/content/drive')

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

def get_soup_with_retries(url, session, retries=3, delay=5):
    """
    Получить содержимое страницы с повторными попытками.
    """
    for attempt in range(retries):
        try:
            response = session.get(url)
            if response.status_code == 200:
                logger.info(f'Успешно получен контент со страницы {url}')
                return BeautifulSoup(response.text, 'html.parser')
            else:
                logger.error(f'Попытка {attempt + 1} из {retries}. Ошибка: {response.status_code}. Повтор через {delay} секунд.')
        except requests.exceptions.RequestException as e:
            logger.error(f'Попытка {attempt + 1} из {retries}. Исключение: {e}. Повтор через {delay} секунд.')
        time.sleep(delay)
    logger.error(f'Не удалось получить содержимое с {url} после {retries} попыток.')
    return None

def save_links_to_file(base_url, file_name, session, start_page=0, end_page=39, single_file=True):
    """
    Сохранить ссылки с вакансий в файл.
    """
    scraped_urls = set()

    for page in range(start_page, end_page + 1):
        logger.info(f"Собираем ссылки со страницы {page} на {base_url}...")
        page_url = f'{base_url}&page={page}'
        soup = get_soup_with_retries(page_url, session)
        if soup:
            links = soup.find_all('a', href=True)
            for link in links:
                href = link['href']
                if 'hh.ru/vacancy/' in href:
                    scraped_urls.add(href)
                    logger.info(f"Найдена ссылка: {href}")
        else:
            logger.warning(f"Не удалось получить данные со страницы {page} на {base_url}.")

    # Запись результатов в файл
    if single_file:
        # Проверяем существование файла перед записью
        file_path = '/content/drive/MyDrive/HH_Vacancy/hh_vacancies.txt'  # Путь на Google Drive

        # Если файл не существует, создаем его
        if not os.path.isfile(file_path):
            open(file_path, 'w').close()

        with open(file_path, 'r+') as file:
            existing_links = set(file.read().splitlines())
            new_links = scraped_urls - existing_links
            file.seek(0, os.SEEK_END)  # Переходим в конец файла для записи новых ссылок
            for url in new_links:
                file.write(url + '\n')
    else:
        # Если нужно записывать в новый файл
        with open(file_name, 'w') as file:
            for url in scraped_urls:
                file.write(url + '\n')

    logger.info(f"Ссылки с {base_url} успешно сохранены в файл {file_name}")

def main():
    """
    Главная функция для сбора ссылок с HeadHunter.
    """
    # Запрашиваем у пользователя ввод Base_URL
    base_url = input("Введите Base_URL для сбора вакансий (например, https://hh.ru/search/vacancy?text=python&area=1): ").strip()

    if not base_url:
        logger.error("Base_URL пустой! Убедитесь, что вы ввели корректный URL.")
        return

    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    })

    file_name = '/content/drive/MyDrive/HH_Vacancy/hh_vacancies.txt'  # Путь к файлу на Google Drive
    save_links_to_file(base_url, file_name, session, start_page=0, end_page=10, single_file=True)

if __name__ == '__main__':
    main()


ЗДЕСЬ ПРОИСХОДИТ СБОР ДАННЫХ ДЛЯ EXEL


In [ ]:
import json
import openpyxl
from openpyxl.utils import get_column_letter
import threading
import queue
import random
import time
import logging
import multiprocessing
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from tqdm import tqdm
from google.colab import drive
from webdriver_manager.chrome import ChromeDriverManager
import datetime

# Монтируем Google Drive
drive.mount('/content/drive')

# Логирование
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(threadName)s - %(levelname)s - %(message)s')

contacts_data = {}

# Функция для импорта cookies
def import_cookies(driver, url, path_to_cookies):
    driver.get(url)
    with open(path_to_cookies, 'r') as cookies_file:
        cookies = json.load(cookies_file)
    for cookie in cookies:
        if 'expiry' in cookie:
            cookie['expiry'] = int(cookie['expiry'])
        if 'sameSite' in cookie and cookie['sameSite'] not in ["Strict", "Lax", "None"]:
            cookie['sameSite'] = "None"
        driver.add_cookie(cookie)
    driver.refresh()

# Функция для получения контактов
def get_contacts(driver, url, xpaths):
    driver.get(url)
    data = {}
    contact_elements = {}

    for key, xpath in xpaths.items():
        try:
            if key == "Показать контакты":
                show_contacts_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, xpath)))
                show_contacts_button.click()

                WebDriverWait(driver, 5).until(
                    lambda d: all(d.find_elements(By.XPATH, xpaths.get("Телефон1", ""))) and
                              all(d.find_elements(By.XPATH, xpaths.get("Почта1", ""))))

                contact_keys = [k for k in xpaths if "Телефон" in k or "Почта" in k]
                for contact_key in contact_keys:
                    contact_elements[contact_key] = driver.find_elements(By.XPATH, xpaths[contact_key])

            elif "Телефон" in key or "Почта" in key:
                if contact_elements.get(key):
                    data[key] = "; ".join([el.text.strip() for el in contact_elements[key] if el.text.strip()])

            elif key == "Профиль компании":
                company_link_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, xpath)))
                company_url = company_link_element.get_attribute('href')
                data[key] = company_url

            else:
                element = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, xpath)))
                if element:
                    data[key] = element.text.strip()
                else:
                    data[key] = "Не найдено"

        except TimeoutException:
            data[key] = "Не найдено"
            logging.error(f"Элемент <<{key}>> не найден по XPath: {xpath}")

    return data

# Функция для сохранения данных в Excel
def save_to_excel(contacts_data , filename):
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = 'HH-Contacts'

    if contacts_data:
        headers = ['URL'] + list(next(iter(contacts_data.values())).keys())
        sheet.append(headers)

        for url , data in contacts_data.items():
            row = [url]
            for key in headers[1:]:
                row.append(data.get(key, 'N/A'))
            sheet.append(row)

        for col_num, column_title in enumerate(headers, 1):
            column_width = max(len(column_title), 20)
            sheet.column_dimensions[get_column_letter(col_num)].width = column_width

        workbook.save(filename)
    else:
        logging.error("Нет данных для сохранения.")

# xpaths_dict, который вы используете для получения данных
xpaths_dict = {
    'Показать контакты': '//button[@data-qa="show-employer-contacts show-employer-contacts_bottom-button"]',
    'Вакансия': '//h1',
    'Компания': '//*[@data-qa="vacancy-company-name"]',
    'Профиль компании': '//span[@class="vacancy-company-name"]/a',
    'Телефон1': '//*[@class="bloko-text bloko-text_small"]/span',
    'Телефон2': '//*[@class="vacancy-contacts-call-tracking__phone-number"]',
    'Почта1': '//*[@data-qa="vacancy-contacts__email"]',
    'Адрес': '//*[@data-qa="vacancy-view-raw-address"]',
    'Зарплата': '//*[@data-qa="vacancy-salary-compensation-type-net"]',
    'Требуемый опыт': '//*[@data-qa="vacancy-experience"]',
    'График': '//*[@data-qa="vacancy-view-employment-mode"]',
    'Описание': '//*[@data-qa="vacancy-description"]',
    'Навыки': '//*[@data-qa="skills-element"]',
}

# Функция для повторения с задержкой при обработке каждого URL
def retry_with_delay(retry_function, max_retries, retry_delay, *args, **kwargs):
    attempt = 0
    while attempt < max_retries:
        try:
            return retry_function(*args, **kwargs)
        except Exception as e:
            attempt += 1
            logging.error(f"Ошибка при попытке {attempt}: {e}")
            if attempt < max_retries:
                time.sleep(retry_delay)
    raise Exception(f"Не удалось выполнить {retry_function.__name__} после {max_retries} попыток")

def worker(path_to_cookies, url_for_auth, user_agents, url_queue, contacts_data_lock, pbar, contacts_data, xpaths_dict, max_retries=3, retry_delay=5, error_urls_queue=None):
    user_agent = random.choice(user_agents)
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--remote-debugging-port=9222")
    chrome_options.add_argument(f"user-agent={user_agent}")

    chromedriver_path = "/usr/local/bin/chromedriver"  # Указываем локальный путь

    chrome_service = Service(chromedriver_path)
    local_driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

    try:
        import_cookies(local_driver, url_for_auth, path_to_cookies)

        while True:
            try:
                current_url = url_queue.get(block=False)
                try:
                    # Обработка URL с повторами
                    contact_data = retry_with_delay(get_contacts, max_retries, retry_delay, local_driver, current_url, xpaths_dict)
                    with contacts_data_lock:
                        contacts_data[current_url] = contact_data
                except Exception as e:
                    logging.error(f"Ошибка при обработке URL {current_url}: {e}")
                    if error_urls_queue is not None:
                        error_urls_queue.put(current_url)
                finally:
                    pbar.update(1)
                    logging.info(f"Processed URL: {current_url}")
                    url_queue.task_done()

                    # Задержка между запросами
                    sleep_time = random.uniform(5, 10)
                    logging.info(f"Задержка перед следующим запросом: {sleep_time:.2f} сек.")
                    time.sleep(sleep_time)

            except queue.Empty:
                logging.info("Очередь обработана.")
                break
    except Exception as e:
        logging.error(f"Неожиданная ошибка: {e}")
    finally:
        local_driver.quit()
        pbar.update(1)
        logging.info("WebDriver закрыт и ресурсы очищены.")


# Главная функция скрипта
def main():
    start_time = time.time()

    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15',
        'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:50.0) Gecko/20100101 Firefox/50.0',
    ]

    path_to_cookies = '/content/drive/MyDrive/HH_Vacancy/hh.json'  # Путь к cookies на Google Drive
    url_for_auth = 'https://hh.ru/'

    contacts_data = {}
    contacts_data_lock = threading.Lock()
    url_queue = queue.Queue()
    error_urls_queue = queue.Queue()

    try:
        with open('/content/drive/MyDrive/HH_Vacancy/hh_vacancies.txt', 'r') as file:
            for url in file:
                url = url.strip()
                if url:
                    url_queue.put(url)
    except IOError as e:
        logging.error(f"Ошибка при чтении файла: {e}")
        return

    pbar = tqdm(total=url_queue.qsize(), desc="Обработка URL")

    num_cores = multiprocessing.cpu_count() or 1
    num_threads = min(10, url_queue.qsize(), num_cores)

    threads = []
    for i in range(num_threads):
        thread = threading.Thread(target=worker, args=(path_to_cookies, url_for_auth, user_agents, url_queue, contacts_data_lock, pbar, contacts_data, xpaths_dict, 3, 5, error_urls_queue))
        threads.append(thread)
        thread.start()

    for t in threads:
        t.join()

    while not error_urls_queue.empty():
        url = error_urls_queue.get()
        try:
            contact_data = retry_with_delay(get_contacts, 3, 5, url, xpaths_dict)
            with contacts_data_lock:
                contacts_data[url] = contact_data
        except Exception as e:
            logging.error(f"Не удалось обработать URL {url} после повторных попыток: {e}")
        pbar.update(1)
        logging.info(f"URL повторно обработан: {url}")

    pbar.close()

    datetime_str = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f'/content/drive/MyDrive/HH_Vacancy/hh-vacancies_{datetime_str}.xlsx'
    save_to_excel(contacts_data, filename)

    end_time = time.time()
    duration = end_time - start_time
    print(f"Сценарий выполнен за {datetime.timedelta(seconds=duration)}")
    logging.info(f"Сценарий выполнен за {datetime.timedelta(seconds=duration)}")

if __name__ == '__main__':
    main()
